In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
dataset.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

dataset['CPM'] = dataset.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
train_data = dataset[(pd.to_datetime(dataset.date) < pd.to_datetime('2019-06-22'))]

In [ ]:
train_data.head()

In [ ]:
test_data = dataset[(pd.to_datetime(dataset.date) >= pd.to_datetime('2019-06-22'))]

In [ ]:
test_data.head()

In [ ]:
test_data = test_data[test_data['CPM'] >= 0]

In [ ]:
test_data = test_data[test_data['CPM'] < test_data['CPM'].quantile(0.95)]

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
dataset.shape

In [ ]:
train_X = train_data.loc[:, train_data.columns != 'CPM']

In [ ]:
train_y = train_data.loc[:, train_data.columns == 'CPM']

In [ ]:
test_X = test_data.loc[:, test_data.columns != 'CPM']

In [ ]:
test_y = test_data.loc[:, test_data.columns == 'CPM']

In [ ]:
train_X = train_X.drop('date', axis=1)

In [ ]:
train_X = train_X.drop('total_revenue', axis=1)

In [ ]:
test_X = test_X.drop('date', axis=1)

In [ ]:
test_X = test_X.drop('total_revenue', axis=1)

In [ ]:
X_train_true, X_valid, y_train_true, y_valid = train_test_split(train_X, train_y, test_size=0.1, random_state=42)

In [ ]:
#Base solution without feature engineering

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from catboost.utils import eval_metric

In [ ]:
params = {'iterations' : 1000,
          'random_seed' : 7,
          'depth'  : 10,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 200
}

In [ ]:
model = CatBoostRegressor(**params)

In [ ]:
model.fit(X_train_true, y_train_true,
          eval_set=(X_valid, y_valid),
          use_best_model=True,
          plot=True
         )

In [ ]:
predictions = model.predict(test_X)

In [ ]:
rmse = eval_metric(test_y['CPM'], predictions, 'RMSE')

In [ ]:
rmse

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(test_y['CPM'], predictions)

In [ ]:
mse

In [ ]:
##

## exp_2

In [ ]:
train_X.head()

In [ ]:
train_X.info()

In [ ]:
train_X.describe()

In [ ]:
categorical_features = ['site_id',
                        'ad_type_id',
                        'ad_unit_id', 
                        'geo_id',
                        'device_category_id',
                        'os_id',
                        'integration_type_id',
                        'advertiser_id',
                        'line_item_type_id'
                       ]

In [ ]:
numeric_features = ['total_impressions',
                     'viewable_impressions',
                     'measurable_impressions'
                    ]

In [ ]:
params = {'iterations' : 100,
          'random_seed' : 7,
          'depth'  : 10,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 200
}

In [ ]:
model_2 = CatBoostRegressor(**params)

In [ ]:
model_2.fit(X_train_true, y_train_true,
            eval_set=(X_valid, y_valid),
            use_best_model=True,
            plot=True, 
            cat_features=list(range(X_train_true[categorical_features].shape[1]))
         )

In [ ]:
predictions = model_2.predict(test_X)

In [ ]:
mse = mean_squared_error(test_y['CPM'], predictions)
mse

## exp_3

In [ ]:
train_X.head()

In [ ]:
X = train_X[categorical_features]

In [ ]:
X.head(5)

In [ ]:
y = train_y

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
params = {'iterations' : 100,
          'random_seed' : 7,
          'depth'  : 10,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 200
}

In [ ]:
model_3 = CatBoostRegressor(**params)

In [ ]:
model_3.fit(x_train, y_train,
            eval_set=(x_valid, y_valid),
            use_best_model=True,
            plot=True, 
            cat_features=list(range(x_train[categorical_features].shape[1]))
         )

In [ ]:
predictions = model_3.predict(test_X[categorical_features])
mse = mean_squared_error(test_y['CPM'], predictions)
mse

## exp_4

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)), x['measurable_impressions'])*1000 , axis=1)

In [ ]:
df = df[df['CPM'] >= 0]

In [ ]:
df = df.drop('total_impressions', axis=1)

In [ ]:
df = df.drop('total_revenue', axis=1)

In [ ]:
df = df.drop('viewable_impressions', axis=1)

In [ ]:
df = df.drop('measurable_impressions', axis=1)

In [ ]:
df = df.drop('revenue_share_percent', axis=1)

In [ ]:
df.head()

In [ ]:
df_train = df[(pd.to_datetime(df.date) < pd.to_datetime('2019-06-22'))]

In [ ]:
df_test  = df[(pd.to_datetime(df.date) >= pd.to_datetime('2019-06-22'))]

In [ ]:
train = df_train[df_train['CPM'] <= np.quantile(df_train['CPM'], 0.95)]

In [ ]:
test = df_test[df_test['CPM'] <= np.quantile(df_test['CPM'], 0.95)]

In [ ]:
X_train = train.drop(columns=['date', 'CPM'])
y_train = train['CPM']

In [ ]:
X_test = test.drop(columns=['date', 'CPM'])
y_test = test['CPM']

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
params_4 = {'iterations' : 100,
          'random_seed' : 7,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 200
}

In [ ]:
model_4 = CatBoostRegressor(**params_4)

In [ ]:
model_4.fit(x_train, y_train,
          eval_set=(x_valid, y_valid),
          cat_features=X_train.columns,
          use_best_model=True,
          plot=True
         )

In [ ]:
predictions = model_4.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse

In [ ]:
params_5 = {'iterations' : 200,
          'random_seed' : 7,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 200,
            'depth' : 15
}

In [ ]:
model_5 = CatBoostRegressor(**params_5)

In [ ]:
model_5.fit(x_train, y_train,
          eval_set=(x_valid, y_valid),
          cat_features=X_train.columns,
          use_best_model=True,
          plot=True
         )

In [ ]:
predictions = model_5.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse

In [ ]:
params_6 = {'iterations' : 100,
          'random_seed' : 7,
          'loss_function' :'RMSE', ## in catbost mse not implemented
          'verbose' : 20,
        }

In [ ]:
model_6 = CatBoostRegressor(**params_6)

In [ ]:
model_6.fit(x_train, y_train,
          eval_set=(x_valid, y_valid),
          cat_features=X_train.columns,
          use_best_model=True,
          plot=True
         )

In [ ]:
predictions = model_6.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse